<a href="https://colab.research.google.com/github/Ajithbalakrishnan/3D-Model-Reconstruction/blob/wipropc/colab_main_attsets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/drive')

<a href="https://colab.research.google.com/github/Ajithbalakrishnan/3D-Model-Reconstruction/blob/wipropc/colab_main_attsets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
cd 3D-Model-Reconstruction/


In [0]:
!unzip 02828884.zip

In [0]:
#!pip install keras==2.2.4
#!pip install pycollada==0.6
!pip install pymcubes==0.0.9    

In [0]:
!pip install keras tensorflow scipy pandas numpy

In [0]:
!git clone https://github.com/Ajithbalakrishnan/3D-Model-Reconstruction.git

In [0]:
import tensorflow as tf
import os
import shutil
import sys
import scipy.io
sys.path.append('..')
import tools as tools
import numpy as np
import time
from tensorflow import keras
from tensorflow.keras import backend
from keras.layers import BatchNormalization,Conv3D,MaxPooling3D,Dense,Reshape,Add,LeakyReLU,Conv3DTranspose,Input, MaxPooling2D, GlobalAveragePooling2D, Dropout, Flatten, Activation, Concatenate, Lambda,Conv2D
from keras.activations import relu,sigmoid,tanh
from keras import models
import copy 
import matplotlib.pyplot as plt 

Using TensorFlow backend.


In [0]:
 tf.reset_default_graph()

In [0]:
batch_size = 2
img_res = 127
vox_res32 = 32
total_mv = 24   
GPU0 = '0'
GPU1 = '0'

re_train=False
#re_train=True             ##############################
single_view_train = True
multi_view_train = True

#####################################

config={}                                 # python dictionary
config['batch_size'] = batch_size
config['total_mv'] = total_mv
config['cat_names'] = ['02933112','04256520']
#config['cat_names'] = ['02691156','02828884','02933112','02958343','03001627','03211117',
#            '03636649','03691459','04090263','04256520','04379243','04401088','04530566']
#config['cat_names'] = ['02828884']
for name in config['cat_names']:
 #content/drive/My Drive/Colab Notebooks/3D-Model-Reconstruction/Data_sample_new/ShapeNetRendering
    config['X_rgb_'+name] = '/content/drive/My Drive/Colab Notebooks/3D-Model-Reconstruction/Data_sample_new/ShapeNetRendering/'+name+'/'
    config['Y_vox_'+name] = '/content/drive/My Drive/Colab Notebooks/3D-Model-Reconstruction/Data_sample_new/ShapeNetVox32/'+name+'/'



In [0]:
def conv2d(x,numfilt,filtsz,stride_=1,pad='same',act=True,name=None):
    x = Conv2D(filters=numfilt, kernel_size=filtsz,strides=stride_,padding=pad,data_format='channels_last',use_bias=False,name=name+'conv2d')(x)
    x = BatchNormalization(axis=3,scale=False,name=name+'conv2d'+'bn')(x)
    if act:
        x = Activation('relu',name=name+'conv2d'+'act')(x)
    return x

def incresA(x,scale,name=None):
    pad = 'same'
    branch0 = conv2d(x,32,1,1,pad,True,name=name+'b0')
    branch1 = conv2d(x,32,1,1,pad,True,name=name+'b1_1')
    branch1 = conv2d(branch1,32,3,1,pad,True,name=name+'b1_2')
    branch2 = conv2d(x,32,1,1,pad,True,name=name+'b2_1')
    branch2 = conv2d(branch2,48,3,1,pad,True,name=name+'b2_2')
    branch2 = conv2d(branch2,64,3,1,pad,True,name=name+'b2_3')
    branches = [branch0,branch1,branch2]
    mixed = Concatenate(axis=3, name=name + '_concat')(branches)
    filt_exp_1x1 = conv2d(mixed,384,1,1,pad,False,name=name+'filt_exp_1x1')
    final_lay = Lambda(lambda inputs, scale: inputs[0] + inputs[1] * scale,
                      output_shape=backend.int_shape(x)[1:],
                      arguments={'scale': scale},
                      name=name+'act_scaling')([x, filt_exp_1x1])
    return final_lay
    
def incresB(x,scale,name=None):
    pad = 'same'
    branch0 = conv2d(x,192,1,1,pad,True,name=name+'b0')
    branch1 = conv2d(x,128,1,1,pad,True,name=name+'b1_1')
    branch1 = conv2d(branch1,160,[1,7],1,pad,True,name=name+'b1_2')
    branch1 = conv2d(branch1,192,[7,1],1,pad,True,name=name+'b1_3')
    branches = [branch0,branch1]
    mixed = Concatenate(axis=3, name=name + '_mixed')(branches)
    filt_exp_1x1 = conv2d(mixed,1152,1,1,pad,False,name=name+'filt_exp_1x1')
    final_lay = Lambda(lambda inputs, scale: inputs[0] + inputs[1] * scale,
                      output_shape=backend.int_shape(x)[1:],
                      arguments={'scale': scale},
                      name=name+'act_scaling')([x, filt_exp_1x1])
    return final_lay
    
def incresC(x,scale,name=None):
    pad = 'same'
    branch0 = conv2d(x,192,1,1,pad,True,name=name+'b0')
    branch1 = conv2d(x,192,1,1,pad,True,name=name+'b1_1')
    branch1 = conv2d(branch1,224,[1,3],1,pad,True,name=name+'b1_2')
    branch1 = conv2d(branch1,256,[3,1],1,pad,True,name=name+'b1_3')
    branches = [branch0,branch1]
    mixed = Concatenate(axis=3, name=name + '_mixed')(branches)
    filt_exp_1x1 = conv2d(mixed,2048,1,1,pad,False,name=name+'fin1x1')
    final_lay = Lambda(lambda inputs, scale: inputs[0] + inputs[1] * scale,
                      output_shape=backend.int_shape(x)[1:],
                      arguments={'scale': scale},
                      name=name+'act_saling')([x, filt_exp_1x1])
    return final_lay
    
def stemblock(img_input):
	x = conv2d(img_input,32,3,2,'valid',True,name='conv1')
	x = conv2d(x,32,3,1,'valid',True,name='conv2')
	x = conv2d(x,64,3,1,'valid',True,name='conv3')

	x_11 = MaxPooling2D(3,strides=1,padding='valid',name='stem_br_11'+'_maxpool_1')(x)
	x_12 = conv2d(x,64,3,1,'valid',True,name='stem_br_12')

	x = Concatenate(axis=3, name = 'stem_concat_1')([x_11,x_12])

	x_21 = conv2d(x,64,1,1,'same',True,name='stem_br_211')
	x_21 = conv2d(x_21,64,[1,7],1,'same',True,name='stem_br_212')
	x_21 = conv2d(x_21,64,[7,1],1,'same',True,name='stem_br_213')
	x_21 = conv2d(x_21,96,3,1,'valid',True,name='stem_br_214')

	x_22 = conv2d(x,64,1,1,'same',True,name='stem_br_221')
	x_22 = conv2d(x_22,96,3,1,'valid',True,name='stem_br_222')

	x = Concatenate(axis=3, name = 'stem_concat_2')([x_21,x_22])

	x_31 = conv2d(x,192,3,1,'valid',True,name='stem_br_31')
	x_32 = MaxPooling2D(3,strides=1,padding='valid',name='stem_br_32'+'_maxpool_2')(x)
	x = Concatenate(axis=3, name = 'stem_concat_3')([x_31,x_32])
	return(x)

def inceptionresnet2(x):
	#Inception-ResNet-A modules
	x = incresA(x,0.15,name='incresA_1')
	x = incresA(x,0.15,name='incresA_2')
	x = incresA(x,0.15,name='incresA_3')
	x = incresA(x,0.15,name='incresA_4')
	print("A modules",x.shape)

	#35 × 35 to 17 × 17 reduction module.
	x_red_11 = MaxPooling2D(3,strides=2,padding='valid',name='red_maxpool_1')(x)

	x_red_12 = conv2d(x,384,3,2,'valid',True,name='x_red1_c1')

	x_red_13 = conv2d(x,256,1,1,'same',True,name='x_red1_c2_1')
	x_red_13 = conv2d(x_red_13,256,3,1,'same',True,name='x_red1_c2_2')
	x_red_13 = conv2d(x_red_13,384,3,2,'valid',True,name='x_red1_c2_3')

	x = Concatenate(axis=3, name='red_concat_1')([x_red_11,x_red_12,x_red_13])
	print("1st reduction modules",x.shape)

	#Inception-ResNet-B modules
	x = incresB(x,0.1,name='incresB_1')
	x = incresB(x,0.1,name='incresB_2')
	x = incresB(x,0.1,name='incresB_3')
	x = incresB(x,0.1,name='incresB_4')
	x = incresB(x,0.1,name='incresB_5')
	x = incresB(x,0.1,name='incresB_6')
	x = incresB(x,0.1,name='incresB_7')
	print("B modules",x.shape)

	#17 × 17 to 8 × 8 reduction module.
	x_red_21 = MaxPooling2D(3,strides=2,padding='valid',name='red_maxpool_2')(x)

	x_red_22 = conv2d(x,256,1,1,'same',True,name='x_red2_c11')
	x_red_22 = conv2d(x_red_22,384,3,2,'valid',True,name='x_red2_c12')

	x_red_23 = conv2d(x,256,1,1,'same',True,name='x_red2_c21')
	x_red_23 = conv2d(x_red_23,256,3,2,'valid',True,name='x_red2_c22')

	x_red_24 = conv2d(x,256,1,1,'same',True,name='x_red2_c31')
	x_red_24 = conv2d(x_red_24,256,3,1,'same',True,name='x_red2_c32')
	x_red_24 = conv2d(x_red_24,256,3,2,'valid',True,name='x_red2_c33')

	x = Concatenate(axis=3, name='red_concat_2')([x_red_21,x_red_22,x_red_23,x_red_24])
	print("2nd reduction modules",x.shape)

	#Inception-ResNet-C modules
	x = incresC(x,0.2,name='incresC_1')
	x = incresC(x,0.2,name='incresC_2')
	x = incresC(x,0.2,name='incresC_3')
	print("C modules",x.shape)

	#TOP
	x = GlobalAveragePooling2D(data_format='channels_last')(x)
	print("avg_pool",x.shape)
	x = Dropout(0.8)(x)
	x = Dense(4096, activation='softmax')(x)
	return(x)

In [0]:
def evaluate_voxel_prediction(prediction, gt):
  #"""  The prediction and gt are 3 dim voxels. Each voxel has values 1 or 0"""
    prediction[prediction >= 0.5] = 1
    prediction[prediction < 0.5] = 0        
    intersection = np.sum(np.logical_and(prediction,gt))
    union = np.sum(np.logical_or(prediction,gt))
    IoU = float(intersection) / float(union)
    return IoU

In [0]:
def refiner_network(volumes_in):
    with tf.device('/gpu:' + GPU1):
        with tf.variable_scope('ref_enc'):
	
            input_volumes_32 = tf.reshape(volumes_in, [-1, vox_res32, vox_res32, vox_res32, 1],name="ref_net_in")
	
            print("input_volumes_32_shape" , input_volumes_32.shape)   #input_volumes_32_shape (?,32,32,32,1)
	
            rn1=Conv3D(filters=32, kernel_size=(4, 4, 4), padding='same',data_format="channels_last",name='ref_c1')(input_volumes_32)
            rn2=BatchNormalization()(rn1)
            rn3=LeakyReLU(alpha=.2)(rn2)
            volumes_16_l =MaxPooling3D(pool_size=(2, 2, 2),name='ref_m1')(rn3)
	
            print("volumes_16_l_shape" , volumes_16_l.shape)      #volumes_16_l_shape (?,16,16,16,32)
	
            rn5=Conv3D(filters=64, kernel_size=(4, 4, 4), padding='same',data_format="channels_last",name='ref_c2')(volumes_16_l)
            rn6=BatchNormalization()(rn5)
            rn7=LeakyReLU(alpha=.2)(rn6)
            volumes_8_l =MaxPooling3D(pool_size=(2, 2, 2),name='ref_m2')(rn7)
		
            print("volumes_8_l_shape" ,volumes_8_l.shape)
		
            rn9=Conv3D(filters=128, kernel_size=(4, 4, 4), padding='same',data_format="channels_last",name='ref_c3')(volumes_8_l)
            rn10=BatchNormalization()(rn9)
            rn11=LeakyReLU(alpha=.2)(rn10)
            volumes_4_l =MaxPooling3D(pool_size=(2, 2, 2),name='ref_m3')(rn11)
		
            print("volumes_4_l_shape" , volumes_4_l.shape)
		
            flatten_features=tf.reshape(volumes_4_l , [-1,8192],name="ref_fc1_in")   
        with tf.variable_scope('ref_fc'):
		
            fc1=Dense(units=2048, activation='relu',name='ref_fc1')(flatten_features)
#            fc1=tanh(fc1)
            
            fc1=relu(fc1, alpha=0.0, max_value=None, threshold=0.0)
		
            print("fc1_shape",fc1.shape)
		
            fc2=Dense(units=8192, activation='relu',name='ref_fc2')(fc1)
#            fc2=tanh(fc2)
            fc2=relu(fc2, alpha=0.0, max_value=None, threshold=0.0)
		
            print("fc2_shape",fc2.shape)
			
            fc2=tf.reshape(fc2, [-1, 4,4,4,128],name="ref_fc2_out")     
		
        with tf.variable_scope('ref_Dec'):
		
            reshaped_1=Add()([fc2,volumes_4_l]) 
		
            print("reshaped_1.shape",reshaped_1.shape)

            rn13=Conv3DTranspose(filters=64, kernel_size=(4, 4, 4), padding='same',data_format="channels_last",name='ref_d1',strides=(2, 2, 2))(reshaped_1)
	
            rn14=BatchNormalization()(rn13)
            volumes_4_r=relu(rn14, alpha=0.0, max_value=None, threshold=0.0)
		
            print("volumes_4_r_shape",volumes_4_r.shape)
		
            reshaped_2=Add() ([volumes_4_r,volumes_8_l]) 
		
            print("reshaped_2_shape",reshaped_2.shape)

	
            rn16=Conv3DTranspose(filters=32, kernel_size=(4, 4, 4), padding='same',data_format="channels_last",name='ref_d2',strides=(2, 2, 2))(reshaped_2)
            rn17=BatchNormalization()(rn16)
            volumes_8_r =relu(rn17, alpha=0.0, max_value=None, threshold=0.0)
		 
            reshaped_3=Add()([volumes_8_r,volumes_16_l])
		
            print("reshaped_3_shape",reshaped_3.shape)
		
	
            rn19=Conv3DTranspose(filters=1, kernel_size=(4, 4, 4), padding='same',data_format="channels_last",name='ref_d3',strides=(2, 2, 2))(volumes_8_r)
            print("rn19_shape",rn19.shape)
#            volumes_16_r= tf.nn.sigmoid(rn19,name='ref_sigmoid1')
#            reshape_4=volumes_16_r                             ####################

            reshape_4=Add()([rn19,input_volumes_32])
            reshape_4=(reshape_4*0.5)
            print("reshape_4_5",reshape_4.shape)
            
            reshape_4= tf.nn.sigmoid(reshape_4,name='ref_sigmoid1')
           
		
            print("reshape_4_sig_shape",reshape_4.shape)
		
            reshape_5=tf.reshape(reshape_4, [-1, vox_res32, vox_res32, vox_res32],name="ref_out")

            return reshape_5
	


In [0]:
def attsets_fc(x, out_ele_num):
	with tf.variable_scope('att_fc'):
		in_ele_num = tf.shape(x)[1]
		in_ele_len = int(x.get_shape()[2])
		out_ele_len = in_ele_len    
		print("out_ele_len ", out_ele_len)
		####################
		x_1st = x
		x_1st_tp = tf.reshape(x_1st, [-1, in_ele_len],name="att_in")
		weights_1st = tools.Ops.fc(x_1st_tp, out_d=out_ele_num*out_ele_len, name="att")
		
		########## option 1
		weights_1st = weights_1st
		########## option 2
#		weights_1st = tf.nn.tanh(weights_1st)

		####################
		weights_1st = tf.reshape(weights_1st, [-1, in_ele_num, out_ele_num, out_ele_len],name="att_fc_out")
		weights_1st = tf.nn.softmax(weights_1st, 1)
		x_1st = tf.tile(x_1st[:,:,None,:], [1,1,out_ele_num,1])
		x_1st = x_1st*weights_1st
		x_1st = tf.reduce_sum(x_1st, axis=1)
		x_1st = tf.reshape(x_1st, [-1, out_ele_num*out_ele_len],name="att_out")       
		return x_1st, weights_1st

#####################################


In [0]:
class Network:
	def __init__(self):
		self.train_mod_dir = './train_mod/'
		self.train_sum_dir = './train_sum/'
		self.test_res_dir = './test_res/'
		self.test_sum_dir = './test_sum/'
		
		print ('re_train : ', re_train)
		if os.path.exists(self.test_res_dir):
			if re_train:
				print ('test_res_dir and files kept!')
			else:
				shutil.rmtree(self.test_res_dir)
				os.makedirs(self.test_res_dir)
				print ('test_res_dir: deleted and then created!')
		else:
			os.makedirs(self.test_res_dir)
			print ('test_res_dir: created!')
		
		if os.path.exists(self.train_mod_dir):
			if re_train:
				if os.path.exists(self.train_mod_dir + 'model.cptk.data-00000-of-00001'):
					print ('model found! will be reused!')
				else:
					print ('model not found! error!')
					#exit()
			else:
				shutil.rmtree(self.train_mod_dir)
				os.makedirs(self.train_mod_dir)
				print ('train_mod_dir: deleted and then created!')
		else:
			os.makedirs(self.train_mod_dir)
			print ('train_mod_dir: created!')
		
		if os.path.exists(self.train_sum_dir):
			if re_train:
				print ('train_sum_dir and files kept!')
			else:
				shutil.rmtree(self.train_sum_dir)
				os.makedirs(self.train_sum_dir)
				print ('train_sum_dir: deleted and then created!')
		else:
			os.makedirs(self.train_sum_dir)
			print ('train_sum_dir: created!')
		
		if os.path.exists(self.test_sum_dir):
			if re_train:
				print ('test_sum_dir and files kept!')
			else:
				shutil.rmtree(self.test_sum_dir)
				os.makedirs(self.test_sum_dir)
				print ('test_sum_dir: deleted and then created!')
		else:
			os.makedirs(self.test_sum_dir)
			print ('test_sum_dir: created!')

	def base_r2n2(self, X_rgb):
		with tf.variable_scope('Encoder'):
			im_num = tf.shape(X_rgb)[1]

			[_, _, d1, d2, cc] = X_rgb.get_shape()
			X_rgb = tf.reshape(X_rgb, [-1, int(d1), int(d2), int(cc)],name="en_in")
			print("Network Structure")
			print("base_r2n2",X_rgb.shape) #base_r2n2 (?, 127, 127, 3)
			with tf.variable_scope('en'):
			
			    x=stemblock(X_rgb)
			    l12=inceptionresnet2(x)
			    print("inception_net",l12.shape)
			    fc = tools.Ops.xxlu(tools.Ops.fc(l12, out_d=4096, name='en_fc1'), label='lrelu')
			    print("fc1_output_r2n",fc.shape)#fc1_output_r2n (?, 1024)
			
		with tf.variable_scope('Att_Net'):	
			#### use fc attention
			input = tf.reshape(fc, [-1, im_num, 4096],name="Att_fc_in")
			print("att_fc_in_r2n",input.shape) #att_fc_in_r2n (?, ?, 4096)
			latent_3d, weights = attsets_fc(input, out_ele_num=1)
			print("att_fc_out_r2n",latent_3d.shape) #att_fc_out_r2n (?, 4096)
			
		with tf.variable_scope('Decoder'):
			####
			latent_3d = tools.Ops.xxlu(tools.Ops.fc(latent_3d, out_d=4*4*4*128, name='de_fc2'), label='lrelu')
			print("fc3_out_r2n",latent_3d.shape) #fc3_out_r2n (?, 8192)
			latent_3d = tf.reshape(latent_3d, [-1, 4, 4, 4, 128],name="de_fc2_out")
		####

			de_c = [128, 128, 128, 64, 32, 1]
			
			print("d1_in_r2n",latent_3d.shape) #d1_in_r2n (?, 4, 4, 4, 128)
			d1 = tools.Ops.xxlu(tools.Ops.deconv3d(latent_3d, k=3, out_c=de_c[1], str=2, name='de_c1'), label='lrelu')
			print("d1_out_r2n",d1.shape) #d1_out_r2n (?, 8, 8, 8, 128)
			d2 = tools.Ops.xxlu(tools.Ops.deconv3d(d1, k=3, out_c=de_c[1], str=1, name='de_c2'), label='lrelu')
			print("d2_out_r2n",d2.shape) #d2_out_r2n (?, 8, 8, 8, 128)
			d00 = tools.Ops.deconv3d(latent_3d, k=1, out_c=de_c[1], str=2, name='de_c00')
			print("d00_out_r2n",d00.shape)#d00_out_r2n (?, 8, 8, 8, 128)
			d2 = d2 + d00
			print("d2+d00_out_r2n",d2.shape)#d2+d00_out_r2n (?, 8, 8, 8, 128)

			d3 = tools.Ops.xxlu(tools.Ops.deconv3d(d2, k=3, out_c=de_c[2], str=2, name='de_c3'), label='lrelu')
			print("d3_out_r2n",d3.shape)#d3_out_r2n (?, 16, 16, 16, 128)
			d4 = tools.Ops.xxlu(tools.Ops.deconv3d(d3, k=3, out_c=de_c[2], str=1, name='de_c4'), label='lrelu')
			print("d4_out_r2n",d4.shape)#d4_out_r2n (?, 16, 16, 16, 128)
			d22 = tools.Ops.deconv3d(d2, k=1, out_c=de_c[2], str=2, name='de_c22')
			print("d22_out_r2n",d22.shape)#d22_out_r2n (?, 16, 16, 16, 128)
			d4 = d4 + d22
			print("d4+d22_out_r2n",d4.shape)#d4+d22_out_r2n (?, 16, 16, 16, 128)

			d5 = tools.Ops.xxlu(tools.Ops.deconv3d(d4, k=3, out_c=de_c[3], str=2, name='de_c5'), label='lrelu')
			print("d5_out_r2n",d5.shape)#d5_out_r2n (?, 32, 32, 32, 64)
			d6 = tools.Ops.xxlu(tools.Ops.deconv3d(d5, k=3, out_c=de_c[3], str=1, name='de_c6'), label='lrelu')
			print("d6_out_r2n",d6.shape)#d6_out_r2n (?, 32, 32, 32, 64)
			d44 = tools.Ops.deconv3d(d4, k=1, out_c=de_c[3], str=2, name='de_c44')
			print("d44_out_r2n",d44.shape)#d44_out_r2n (?, 32, 32, 32, 64)
			d6 = d6 + d44
			print("d6+d44_out_r2n",d6.shape) #d6+d44_out_r2n (?, 32, 32, 32, 64)

			d7 = tools.Ops.xxlu(tools.Ops.deconv3d(d6, k=3, out_c=de_c[4], str=1, name='de_c7'), label='lrelu')
			print("d7_out_r2n",d7.shape) #d7_out_r2n (?, 32, 32, 32, 32)
			d8 = tools.Ops.xxlu(tools.Ops.deconv3d(d7, k=3, out_c=de_c[4], str=1, name='de_c8'), label='lrelu')
			print("d8_out_r2n",d8.shape)#d8_out_r2n (?, 32, 32, 32, 32)
			d77 = tools.Ops.xxlu(tools.Ops.deconv3d(d7, k=3, out_c=de_c[4], str=1, name='de_c77'), label='lrelu')
			print("d77_out_r2n",d77.shape)#d77_out_r2n (?, 32, 32, 32, 32)
			d8 = d8 + d77
			print("d8+d77_out_r2n",d8.shape) #d8+d77_out_r2n (?, 32, 32, 32, 32)

			d11 = tools.Ops.deconv3d(d8, k=3, out_c=de_c[5], str=1, name='de_c9')
			print("d11_out_r2n",d11.shape) #d11_out_r2n (?, 32, 32, 32, 1)
			
			ref_in = tf.reshape(d11, [-1, vox_res32, vox_res32, vox_res32],name="ref_in")     ###
			
			y = tf.nn.sigmoid(d11,name='de_sigmoid')

			att_o = tf.reshape(y, [-1, vox_res32, vox_res32, vox_res32],name="de_out")
			
			print("att_out_shape",att_o.shape) #att_out_shape (?, 32, 32, 32)
			
		with tf.variable_scope('ref_net'):
		
			ref_o=refiner_network(ref_in)
			
			return ref_o,att_o, weights

	def build_graph(self):
		img_res = 127
		vox_res = 32
		self.X_rgb = tf.placeholder(shape=[None, None, img_res, img_res, 3], dtype=tf.float32)
		self.Y_vox = tf.placeholder(shape=[None, vox_res, vox_res, vox_res], dtype=tf.float32)
		self.lr = tf.placeholder(tf.float32)
		self.refine_lr = tf.placeholder(tf.float32)
		with tf.device('/gpu:' + GPU0):
			self.Y_pred,self.vae_o, self.weights = self.base_r2n2(self.X_rgb)
			tf.summary.histogram('Attsets_Weights', self.weights)
		with tf.device('/gpu:' + GPU1):	
			### rec loss
			print ("reached")
			with tf.variable_scope('Loss_Fun'):
				Y_vox_ = tf.reshape(self.Y_vox, shape=[-1, vox_res ** 3])
				Y_pred_ = tf.reshape(self.Y_pred, shape=[-1, vox_res ** 3])
				vae_o_=tf.reshape(self.vae_o, shape=[-1, vox_res ** 3])
			
				self.vae_loss = tf.reduce_mean(-tf.reduce_mean(Y_vox_ * tf.log(vae_o_ + 1e-8), reduction_indices=[1]) -
										 tf.reduce_mean((1 - Y_vox_) * tf.log(1 - vae_o_ + 1e-8),reduction_indices=[1]))
				self.rec_loss = tf.reduce_mean(-tf.reduce_mean(Y_vox_ * tf.log(Y_pred_ + 1e-8), reduction_indices=[1]) -
										 tf.reduce_mean((1 - Y_vox_) * tf.log(1 - Y_pred_ + 1e-8),reduction_indices=[1]))
				sum_rec_loss = tf.summary.scalar('rec_loss', self.rec_loss)
				self.sum_merged = sum_rec_loss
				tf.summary.histogram('rec_loss', self.rec_loss)
				tf.summary.scalar("vae-loss",self.vae_loss)
				tf.summary.histogram("vae_loss",self.vae_loss)
				self.mean_loss = tf.div(x=tf.math.add(x=self.vae_loss,y=self.rec_loss,name='add_loss'),y=2,name='mean_loss')
				tf.summary.histogram("mean_vae_loss",self.mean_loss)
				tf.summary.scalar("mean_vae_loss",self.mean_loss)
				
			with tf.variable_scope('Evaluation_Metric'):			    
                		gt_vox=Y_vox_
                		self.iou_ref = metric_iou(Y_pred_,gt_vox)
                		tf.summary.scalar('iou_refiner', self.iou_ref)
                		tf.summary.histogram('iou_refiner', self.iou_ref)
                		self.iou_vae = metric_iou(vae_o_,gt_vox)
                		tf.summary.scalar('iou_vae', self.iou_vae)
                		tf.summary.histogram("iou_vae",self.iou_vae)
                		
			with tf.variable_scope('Optimization'):

                		base_en_var = [var for var in tf.trainable_variables() if var.name.startswith('Encoder/en')]
                		base_dec_var = [var for var in tf.trainable_variables() if var.name.startswith('Decoder/de')]
                		att_var = [var for var in tf.trainable_variables() if var.name.startswith('Att_Net/att')]
                		refine_var = [var for var in tf.trainable_variables() if var.name.startswith('ref_net/ref')]
                		self.refine_optim = tf.train.AdamOptimizer(learning_rate=self.refine_lr).minimize(self.rec_loss, var_list=refine_var)
                		self.base_en_optim2 = tf.train.AdamOptimizer(learning_rate=self.lr).minimize(self.mean_loss, var_list=base_en_var)
                		self.base_de_optim2 = tf.train.AdamOptimizer(learning_rate=self.lr).minimize(self.mean_loss, var_list=base_dec_var)
                		self.att_optim2 = tf.train.AdamOptimizer(learning_rate=self.lr).minimize(self.mean_loss, var_list=att_var)
			    				
		
		print ("total weights:",tools.Ops.variable_count())
		self.saver = tf.train.Saver(max_to_keep=1)
		config = tf.ConfigProto(allow_soft_placement=True)
		config.gpu_options.allow_growth = True
		config.gpu_options.visible_device_list = '0'
		self.sess = tf.Session(config=config)
		self.merged = tf.summary.merge_all()
		self.sum_writer_train = tf.summary.FileWriter(self.train_sum_dir, self.sess.graph)
		self.sum_writer_test = tf.summary.FileWriter(self.test_sum_dir, self.sess.graph)

		#######################
		path = self.train_mod_dir
		#path = './Model_released/'  # retrain the released model

		if os.path.isfile(path + 'model.cptk.data-00000-of-00001'):
			print ("restoring saved model!")
			self.saver.restore(self.sess, path + 'model.cptk')
		else:
			self.sess.run(tf.global_variables_initializer())
		return 0
    
	def train(self, data):
		for epoch in range(0, 250, 1):
			train_view_num = 24  ##!!!!!!!!!!!
			data.shuffle_train_files(epoch, train_mv=train_view_num)
			total_train_batch_num = data.total_train_batch_num  #int(len(self.X_rgb_train_files)/(self.batch_size*train_mv))
			print ('total_train_batch_num:', total_train_batch_num)
			for i in range(total_train_batch_num):
				#### training
				X_rgb_bat, Y_vox_bat = data.load_X_Y_train_next_batch(train_mv=train_view_num)
				print("multi_view_train_X_rgb_bat : ",X_rgb_bat.shape)#np.asarray(X.append(X_rgb[b*train_mv:(b+1)*train_mv,:]))
				

				print(time.ctime())
				
				##### option 1: seperate train, seperate optimize
				if epoch<=30:
					single_view_train=True
					multi_view_train=False
				else:
					single_view_train=False
					multi_view_train=True

				##### optiion 2: joint train, seperate optimize
				#single_view_train = True
				#multi_view_train = True
				
				if epoch <= 10:
					att_lr=.001
					ref_lr=.001
				if epoch > 10 and epoch <= 20:
					att_lr=.0003
					ref_lr=.0003
				if epoch > 20 :
					att_lr=.0001

				###########  single view train
				if single_view_train:
					
					rgb = np.reshape(X_rgb_bat,[batch_size*train_view_num, 1, 127,127,3])
					print("single_view_train_rgb_input_shape ",rgb.shape)
					vox = np.tile(Y_vox_bat[:,None,:,:,:],[1,train_view_num,1,1,1])
					vox = np.reshape(vox, [batch_size*train_view_num, 32,32,32])	
					vae_loss_c,eee,ddd, rec_loss_c, sum_train,rrr,mean_vae,iou_ref_,iou_vae_ = self.sess.run([self.vae_loss,self.base_en_optim2,self.base_de_optim2,self.rec_loss,self.merged,self.refine_optim,self.mean_loss,self.iou_ref,self.iou_vae],feed_dict={self.X_rgb: rgb, self.Y_vox: vox, self.lr: att_lr,self.refine_lr: ref_lr})
					print ('ep:', epoch, 'i:', i, 'train single rec loss:', rec_loss_c)
					print ('ep:', epoch, 'i:', i, 'train single vae loss:', vae_loss_c)
					print ('ep:', epoch, 'i:', i, 'train single mean_vae loss:',mean_vae)
					print ('ep:', epoch, 'i:', i, 'train single ref_iou:',iou_ref_)
					print ('ep:', epoch, 'i:', i, 'train single vae_iou:',iou_vae_)
                                        									
				########## multi view train
				if multi_view_train:
					
					vae_loss_c,rec_loss_c, _, sum_train,xxx,mean_vae,iou_ref_,iou_vae_ = self.sess.run([self.vae_loss,self.rec_loss, self.att_optim2, self.merged,self.refine_optim,self.mean_loss,self.iou_ref,self.iou_vae],feed_dict={self.X_rgb: X_rgb_bat, self.Y_vox: Y_vox_bat,self.lr: att_lr,self.refine_lr: ref_lr})
					print ('ep:', epoch, 'i:', i, 'train multi rec loss:', rec_loss_c)
					print ('ep:', epoch, 'i:', i, 'train multi vae loss:', vae_loss_c)
					print('ep:',epoch,'i',i,'train multi mean_vae loss:',mean_vae)
					print ('ep:', epoch, 'i:', i, 'train multi ref_iou:',iou_ref_)
					print ('ep:', epoch, 'i:', i, 'train multi vae_iou:',iou_vae_)
                                        				
				############
				if i % 10 == 0:
					self.sum_writer_train.add_summary(sum_train, epoch * total_train_batch_num + i)
					
				
				#### testing
				if i % 50 == 0 :
					X_rgb_batch, Y_vox_batch = data.load_X_Y_test_next_batch(test_mv=3)
					
#					vae_pred = tf.get_default_graph().get_tensor_by_name("Decoder/de_out:0")
#					ref_pred = tf.get_default_graph().get_tensor_by_name("ref_net/ref_Dec/ref_out:0")
#					gt_vox=Y_vox_batch.astype(np.float32)
					
#					iou_pred = metric_iou(ref_pred,gt_vox)
#					tf.summary.scalar("iou",iou_pred)

					rrrr,aaaa,rec_loss_te, qwerty, Y_vox_test_pred, att_pred, sum_test,mean_vae,iou_ref_,iou_vae_ = \
						self.sess.run([self.refine_optim,self.att_optim2,self.rec_loss,self.vae_loss, self.Y_pred,self.weights, self.merged,self.mean_loss,self.iou_ref,self.iou_vae],feed_dict={self.X_rgb: X_rgb_batch, self.Y_vox: Y_vox_batch,self.lr: att_lr,self.refine_lr: ref_lr})
						
					X_rgb_batch = X_rgb_batch.astype(np.float16)
					
					Y_vox_batch = Y_vox_batch.astype(np.float16)
					Y_vox_test_pred = Y_vox_test_pred.astype(np.float16)
					att_pred = att_pred.astype(np.float16)
					to_save = {'X_test':X_rgb_batch,'Y_test_pred':Y_vox_test_pred,'att_pred':att_pred,'Y_test_true':Y_vox_batch}
					scipy.io.savemat(self.test_res_dir+'X_Y_pred_'+str(epoch).zfill(2)+'_'+str(i).zfill(5)+'.mat',to_save,do_compression=True)
					
					
					self.sum_writer_test.add_summary(sum_test, epoch * total_train_batch_num + i)
										
#					iou_ref=evaluate_voxel_prediction(ref_pred_,gt_vox)
#					iou_vae=evaluate_voxel_prediction(vae_pred_,gt_vox)
					
#					print("Ref_iou:",iou_ref)
#					print("Vae_iou:",iou_vae)
					
#					plot_list_iou.append(iou_ref)
#					plot_list_i.append((i/50))
#					graph_plot(plot_list_iou,plot_list_i)
					print ('ep:', epoch, 'i:', i, 'test rec loss:', rec_loss_te)
					print ('ep:', epoch, 'i:', i, 'test vae loss:', qwerty )
					print ('ep:', epoch, 'i:', i, 'test mean_vae loss:', mean_vae) 
					print ('ep:', epoch, 'i:', i, 'test ref_iou:',iou_ref_)
					print ('ep:', epoch, 'i:', i, 'test vae_iou:',iou_vae_)
					
				#### model saving
				if i % 100 == 0 :
					self.saver.save( self.sess, save_path=self.train_mod_dir + 'model.cptk' )
					print ( 'epoch:', epoch, 'i:', i, 'model saved!' )
#					plt.show()
				

									

##########

In [0]:
if __name__ =='__main__':

		net = Network()          #net=object to create instance

		print("network compleated")   ###

		net.build_graph()
		print("graph compleated")
                
#               sys.exit(). sys.exit()        ###
		
		data = tools.Data(config)
		print("tools.data compleated")

                
		print('trianing data')
		
		net.train(data)